In [ ]:
# !pip install dvc mlflow

In [1]:
# !npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 2.02s


In [2]:
!dvc get https://github.com/iterative/dataset-registry tutorials/versioning/data.zip

In [ ]:
!unzip data.zip

In [ ]:
!dvc add data


To track the changes with git, run:

	git add data.dvc

To enable auto staging, run:

	dvc config core.autostage true


In [ ]:
!git add data.dvc .gitignore

In [4]:
import numpy as np
import sys
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import applications
from tensorflow.keras.callbacks import CSVLogger
from tqdm.keras import TqdmCallback
from tensorflow import keras 
from PIL import Image
import time 

import mlflow
from pathlib import Path

In [5]:
experiment_id = mlflow.create_experiment(
    "cat-dog VGG classifier", 
    artifact_location=Path.cwd().joinpath("./mlruns/mlruns.db").as_uri(),
    tags={"version":"v1", "priority":"P1"}
)

In [6]:
mlflow.set_tracking_uri("sqlite:///mlruns/mlruns.db")

In [7]:
experiment = mlflow.set_experiment(experiment_name="cat-dog VGG classifier")

2022/09/20 09:41:24 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2022/09/20 09:41:24 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [8]:
experiment

<Experiment: artifact_location='./mlruns/1', creation_time=1663666884733, experiment_id='1', last_update_time=1663666884733, lifecycle_stage='active', name='cat-dog VGG classifier', tags={}>

In [9]:
path = os.getcwd()

In [10]:
img_width, img_height = 150, 150

In [11]:
top_model_weights_path = 'model.h5'
train_data_dir = os.path.join('data', 'train')
validation_data_dir = os.path.join('data', 'validation')
cats_train_path = os.path.join(path, train_data_dir, 'cats')
nb_train_samples = 2 * len([name for name in os.listdir(cats_train_path)
                            if os.path.isfile(
                                os.path.join(cats_train_path, name))])


In [12]:
batch_size = 32

In [13]:
datagen = ImageDataGenerator(rescale=1. / 255)



train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True)

val_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)

Found 1000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [14]:
# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

58900480/58889256 [==============================] - 0s 0us/step


In [15]:
model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))
x = model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)

outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)


In [16]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________


In [17]:
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.09, nesterov=False)
# learning schedulre
epochs = 50

In [18]:
model.compile(optimizer=optimizer,loss=loss_fn)

In [19]:
acc_metric = tf.keras.metrics.BinaryAccuracy()

In [ ]:
with mlflow.start_run(run_name="exp-epcoh-20",experiment_id=experiment.experiment_id, description="A model to classify between cat and dog example") as run:

    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("objective", "BinaryCrossentropy")
    mlflow.log_param("epochs", epochs)
    
    for epoch in range(epochs):
        print(f"Epoch: {epoch}")  
        for step, (inputs, targets) in zip(range(len(train_generator)), train_generator):
            
            with tf.GradientTape() as tape:
                # Forward pass.
                logits = model(inputs)
                # Compute the loss value for this batch.
                targets = targets.reshape(logits.shape)
                # targets = np.expand_dims(targets, axis=0)
                loss_value = loss_fn(targets, logits)
            
            acc_metric.update_state(targets, logits)


            # Get gradients of loss wrt the *trainable* weights.
            gradients = tape.gradient(loss_value, model.trainable_weights)
            # Update the weights of the model.
            optimizer.apply_gradients(zip(gradients, model.trainable_weights))
            
            
            if step % 30 == 0:
              print(
                  f"Training loss at step {step}: {loss_value:.4f}"
              )
            mlflow.log_metric("Training loss", loss_value)
            
        train_acc = acc_metric.result().numpy()

        print(f"Training acc over epoch: {train_acc:.4f}")
            
        mlflow.log_metric("training accuracy", train_acc)
        
        print("Validating...")
        val_acc = model.evaluate(val_generator)

        mlflow.log_metric("validation accuracy", val_acc)
        print(f"Validation acc: {val_acc:.4f}")
        
    mlflow.keras.log_model(model, experiment.artifact_location, registered_model_name="cat-dog VGG classifier")
    os.makedirs("artifact_data", exist_ok=True)
    model.save_weights("artifact_data/model_weights.h5")
    model.save("artifact_data/keras_model")
    # with open("artifact_data/model_summary.txt", 'w', encoding='utf-8') as fs:
    #     fs.write(model.summary())
        
    mlflow.log_artifacts("artifact_data", artifact_path="artifacts")

Epoch: 0
Training loss at step 0: 0.7037


In [25]:
get_ipython().system_raw("mlflow server --host 0.0.0.0 -p 8000 --backend-store-uri sqlite:///mlruns/mlruns.db --default-artifact-root ./mlruns &")

In [26]:
get_ipython().system_raw('lt --port 8000 >> url.txt 2>&1 &')

In [24]:
!rm url.txt

In [27]:
!cat url.txt

your url is: https://silver-bats-guess-35-196-3-165.loca.lt


In [ ]:
!dvc add model.h5 metrics.csv

## Serving a registered model

In [ ]:
import mlflow.pyfunc

In [ ]:
model_name = "VGG cat-dog classifier"
stage = 'Staging'

In [ ]:
model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{stage}"
)

In [ ]:
test_img = next(val_generator)

In [ ]:
img_name = "dogs\\dog.6.jpg"

In [ ]:
test_img = np.array(Image.open(os.path.join(train_data_dir, img_name)))
test_img = np.expand_dims(test_img, axis=0)

In [ ]:
test_img.shape

(1, 488, 499, 3)

In [ ]:
model.predict(test_img)